### MULTINOMIAL
 > - 분류 범위가 여러개일 때 (0,1,2는 숫자를 나타내지 않음 범주임)
 > - cost function => cross-entropy-ftn 의 일부
 > - Y 값을 one-hot방법으로 사용하는 이유 : 0,1,2로 분류하게 되면 1.5가 나왔을 때 애매


### softmax 
 > - MULTINOMIAL에서 sigmoid 함수 대신 사용
 > - softmax대신 sigmoid 함수를 사용하면 Y벡터의 결과값은 서로 독립적이기 때문에 분류에 애매한 경우가 있다
 > - softmax를 적용하면 결과의 총합이 1로, 스코어값이 마치 각 클래스에 분류될 확률처럼 보이는 결과

In [1]:
# Lab 6 Softmax Classifier
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1], # C클래스 -> ont hot 방법
          [0, 0, 1], # C클래스
          [0, 0, 1],
          [0, 1, 0], # B클래스로 분류
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3]) 
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
# prediction modeling
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
 
# Cross entropy cost/loss
# Y벡터 1행에 대해 sum이 필요(element의 곱 , not 행렬 곱)
# 적합한 modeling을 구하기 위한 최소화
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1)) 

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    print('--------------')

    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, axis=1))) # 행에서 최대값의 index return

    print('--------------')

    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, axis=1)))

    print('--------------')

    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')

    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1))) # 각각의 행에서 max index 리턴

0 5.048624
200 0.5484252
400 0.43214852
600 0.3524663
800 0.2768294
1000 0.23268294
1200 0.21055308
1400 0.19221492
1600 0.17675184
1800 0.16353412
2000 0.15210836
--------------
[[1.3890432e-03 9.9860197e-01 9.0612402e-06]] [1]
--------------
[[0.93119204 0.06290206 0.00590588]] [0]
--------------
[[1.2732767e-08 3.3411323e-04 9.9966586e-01]] [2]
--------------
[[1.3890432e-03 9.9860197e-01 9.0612402e-06]
 [9.3119204e-01 6.2902056e-02 5.9058843e-03]
 [1.2732767e-08 3.3411323e-04 9.9966586e-01]] [1 0 2]


### fancy softmax(multinomial) classification

In [2]:
# before
# cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1)) 

logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)
 # Cross entropy cost/loss
cross_entropy_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot) # L(i)*S(i)
cost = tf.reduce_mean(cross_entropy_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

NameError: name 'logits' is not defined

### Animal classification

In [5]:
 # Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('../tensorflow/data/data-04-zoo.csv', delimiter=',', dtype=np.float32) 
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6의 숫자를 줘야함 [1,0,0,0,0,0] => 0이어야함
# rank는 차원을 말해줌, shape는 몇 행 몇 열인지
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6, shape(?,1)
Y_one_hot = tf.one_hot(Y, nb_classes)  # Y값을 one hot으로 바꿔줘야함, shape(?,1,7) 3차원
print("one_hot", Y_one_hot) 
   # error : one_hot은 1차원 더해짐 [[0],[1]] => [[[1,0,0,0,0,0]],[[0,1,0,0,0,0]]]
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?,7)
print("reshape", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
 # Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                 labels=tf.stop_gradient([Y_one_hot]))
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
prediction = tf.argmax(hypothesis, 1) # 0~6값 -> one_hot 이전값으로 됨
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step:     0	Loss: 3.796	Acc: 31.68%
Step:   100	Loss: 0.651	Acc: 79.21%
Step:   200	Loss: 0.416	Acc: 87.13%
Step:   300	Loss: 0.307	Acc: 92.08%
Step:   400	Loss: 0.245	Acc: 95.05%
Step:   500	Loss: 0.204	Acc: 95.05%
Step:   600	Loss: 0.175	Acc: 97.03%
Step:   700	Loss: 0.154	Acc: 98.02%
Step:   800	Loss: 0.137	Acc: 98.02%
Step:   900	Loss: 0.123	Acc: 98.02%
Step:  1000	Loss: 0.112	Acc: 99.01%
Step:  1100	Loss: 0.102	Acc: 99.01%
Step:  1200	Loss: 0.094	Acc: 100.00%
Step:  1300	Loss: 0.088	Acc: 100.00%
Step:  1400	Loss: 0.082	Acc: 100.00%
Step:  1500	Loss: 0.076	Acc: 100.00%
Step:  1600	Loss: 0.072	Acc: 100.00%
Step:  1700	Loss: 0.068	Acc: 100.00%
Step:  1800	Loss: 0